initializes a neural network using Keras, with a single dense layer, and imports necessary mathematical and array manipulation libraries.

In [ ]:
import numpy as np
from math import pi, cos
from keras.models import Sequential
from keras.layers import Dense

This code defines variables for a simulation or calculation: the final time (TFinal), the time step (Dt), the constant K, the efficiency (Eff) of a system or process, and the maximum power output (PtMaxPanel) of a solar panel or similar device.

In [ ]:
TFinal = 97.0
Dt = 1.0
K = 191.1
Eff = 21.528 / 100
PtMaxPanel = 24.24


This function calculates the available power at a given time, adjusting for phase, efficiency, and a scaling factor, ensuring it never goes negative. It then generates a series of power values over specified time intervals, incorporating a random phase shift.

In [ ]:
def available_power(t, phase, k, efficiency, pt_max_panel):
    return max(pt_max_panel * cos(2 * pi * t / TFinal + phase) * efficiency * k, 0)

phase = np.random.rand() * 2 * pi
times = np.arange(0, TFinal + Dt, Dt)
powers = [available_power(t, phase, K, Eff, PtMaxPanel) for t in times]

This function computes the first and second numerical derivatives of a dataset, using finite difference methods, given the dataset and time step size.

In [ ]:
def calculate_derivatives(data, dt):
    d1 = np.zeros(len(data))
    d2 = np.zeros(len(data))
    for i in range(2, len(data)):
        d1[i] = (data[i] - data[i - 1]) / dt
        if i > 2:
            d2[i] = (data[i] - 2 * data[i - 1] + data[i - 2]) / (dt ** 2)
    return d1, d2

The function label_data calculates the time until the next charging event for each time point in a power data array, working backwards and using a specified time interval dt.

In [ ]:
def label_data(powers, dt):
    labels = np.zeros(len(powers))
    countdown = 0
    charging = powers[0] > 0

    for i in reversed(range(len(powers))):
        if powers[i] > 0:
            charging = True
            countdown = 0
        elif charging:
            charging = False
            countdown = 0
        else:
            countdown += dt

        labels[i] = countdown
    return labels

In [ ]:
labels = label_data(powers, Dt)
socs = np.cumsum(powers)
soc_derivs, soc_derivs2 = calculate_derivatives(socs, Dt)
features = np.vstack([soc_derivs, soc_derivs2]).T
features = features[2:]
labels = labels[2:]

model = Sequential([
    Dense(64, input_dim=2, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(features, labels, epochs=10, batch_size=1, verbose=0)
test_index = 30  # Un momento cualquiera
test_features = np.array([[soc_derivs[test_index], soc_derivs2[test_index]]])
predicted_steps = model.predict(test_features)
print(f"Pasos predichos hasta el próximo ciclo de carga: {predicted_steps[0][0]}")

1/1 [==============================] - 0s 75ms/step
Pasos predichos hasta el próximo ciclo de carga: 22.081836700439453
